In [1]:
import logging
import os
import pathlib
import subprocess
import warnings
from glob import glob

import earthpy.appeears as etapp
import folium
import geopandas as gpd
import hvplot.pandas
import hvplot.xarray
import pandas as pd
import rioxarray as rxr
import xarray as xr

# Set up logging so AppeearsDownloader will log in notebook
logging.basicConfig(level=logging.INFO)

# Ignore FutureWarning coming from hvplot
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# Create the directory where we will download data for the project
data_path = os.path.join(
    pathlib.Path.home(),
    'earth-analytics',
    'data',
    'dixie-fire')
os.makedirs(data_path, exist_ok=True)
data_path

'/home/jovyan/earth-analytics/data/dixie-fire'

In [3]:
# Download the Dixie Fire boundary spatial data and create a geodataframe

dixie_url = ('https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis/rest/'
             'services/InterAgencyFirePerimeterHistory_All_Years_View/'
             'FeatureServer/0/query?where=incident=%27DIXIE%27%20AND%20'
             'FIRE_YEAR_INT=2021%20AND%20SOURCE%20=%20%27CalFire%20FRAP'
             '%27&outFields=*&outSR=4326&f=json'
)

dixie_df = gpd.read_file(dixie_url)

dixie_df

,OBJECTID,MAP_METHOD,DATE_CUR,COMMENTS,GEO_ID,IRWINID,UNQE_FIRE_ID,FIRE_YEAR,LOCAL_NUM,INCIDENT,...,POO_RESP_I,FEATURE_CA,GIS_ACRES,USER_NAME,SOURCE,AGENCY,FIRE_YEAR_INT,Shape__Area,Shape__Length,geometry
0,11634,Mixed Collection Methods,202107140015,"E9DB628D-DFB7-4C85-BCD6-1254574D946B, contains...",43182,NaN,2021-CABTU-00009205,2021,00009205,DIXIE,...,NaN,Wildfire,963405.4,NaN,CalFire FRAP,CDF,2021,6.692866e+09,2.010262e+06,"MULTIPOLYGON (((-121.08903 40.32211, -121.0890..."


In [4]:
# Create a map using Folium and the Camp Fire boundary data we downloaded
dixie_lat = 40.13775
dixie_lon = -121.06855

m = folium.Map(
    location=(dixie_lat, dixie_lon),
    zoom_start=10,
    scrollWheelZoom=False,
    tiles = ("https://server.arcgisonline.com/ArcGIS/rest/services/"
             "NatGeo_World_Map/MapServer/tile/{z}/{y}/{x}"
            ),
    attr = ("Tiles &copy; Esri &mdash; National Geographic, Esri,"
            "DeLorme, NAVTEQ, UNEP-WCMC, USGS, NASA, ESA, METI, NRCAN,"
            "GEBCO, NOAA, iPC'"
            )
    
    )

# Add polygon of Dixie Fire, add labels and marker for centroid of fire
for idx, row in dixie_df.iterrows():
    # Add polygon for Dixie Fire, add popup label
    
    poly_style = lambda feature: {"fillcolor": "#ffff00","color":"red"}
    poly = folium.GeoJson(row['geometry'],poly_style)
    label = folium.Popup(row['INCIDENT'])
    label.add_to(poly)
    poly.add_to(m)
    # create and add marker for centroid
    folium.Marker([dixie_lat, dixie_lon], popup="Dixie Fire").add_to(m)

m

In [5]:
# Delete wrong files

# del_data_path = data_path + "/modis-ndvi/MYD13Q1.061_2017137_to_2022243"

# for filename in os.listdir(del_data_path):
#     file_path = os.path.join(del_data_path, filename)
#     if os.path.isfile(file_path):
#         os.remove(file_path)

In [6]:


# Initialize AppeearsDownloader for MODIS NDVI data
ndvi_downloader = etapp.AppeearsDownloader(
    download_key='modis-ndvi',
    ea_dir=data_path,
    product='MYD13Q1.061',
    layer='_250m_16_days_NDVI',
    start_date='06-01',
    end_date='08-31',
    recurring=True,
    year_range=[2020, 2023],
    polygon=dixie_df
)

# Download files if the download directory does not exist
if not os.path.exists(ndvi_downloader.data_dir):
    ndvi_downloader.download_files()
ndvi_downloader



INFO:root:Submitting task request...
INFO:root:Logging into AppEEARS API...
INFO:root:Login successful. Auth Header: Bearer 4aAsEeK9FRAVRYtWthSB52QQtMS-gDwSjMmhLAig359zQosuWVmOVFbU32HQYRBtoYp5iC847qseQjdkkfDbHw
INFO:root:task request successfully completed
INFO:root:Submitting status/{task_id} request...
INFO:root:status/{task_id} request successfully completed
INFO:root:queued
INFO:root:Submitting status/{task_id} request...
INFO:root:status/{task_id} request successfully completed
INFO:root:5
INFO:root:Submitting status/{task_id} request...
INFO:root:status/{task_id} request successfully completed
INFO:root:53
INFO:root:Submitting status/{task_id} request...
INFO:root:status/{task_id} request successfully completed
INFO:root:75
INFO:root:Submitting status/{task_id} request...
INFO:root:status/{task_id} request successfully completed
INFO:root:done
INFO:root:Task completed - ready for download.
INFO:root:Current task status: done
INFO:root:Submitting bundle/{task_id} request...
INFO:r